Decode
------

<div style="text-align: justify"> Dataset at Kaggle <a href="https://www.kaggle.com/datasets/aw6ro7zcd/shells">(click)</a> have to be decoded before any use. All these proceeding below is tightly related with the proceeding of encode that can be found at Github <a href="https://github.com/AW6RO7ZCD/kaggle-shells/blob/master/clean_and_encode.ipynb">(click)</a>.</div>
</br>
<div style="text-align: justify"> The same code, but as a .py file can be found at the same directory at Github <a href="https://github.com/AW6RO7ZCD/kaggle-shells/blob/master/clean_and_encode.ipynb">(click)</a></div>
</br>
<div style="text-align: justify">To decode, go the following:</div>

In [1]:
# Python 3.11.0

In [2]:
import pandas as pd

In [3]:
data: pd.DataFrame = pd.DataFrame(columns=['Brightness',
                                           'Orientation',
                                           'Stripes',
                                           'AntiStripes',
                                           'CornerAngle',
                                           'DilationAngle',
                                           'Length',
                                           'Width',
                                           'Height'])

In [4]:
ANGLE_INTERVAL: float = 5.625

In [5]:
def decode(column: str, code: str) -> object:
    encoded: int = int(code,2)
    nan_value = pd.to_numeric(None)
    
    match column:
        case 'Brightness':
            match encoded:
                case 0:
                    return nan_value
                case _:
                    return encoded - 1
        case 'Orientation':
            return encoded
        case 'Stripes':
            match encoded:
                case 0:
                    return nan_value
                case _:
                    return encoded - 1
        case 'AntiStripes':
            match encoded:
                case 0:
                    return nan_value
                case _:
                    return encoded - 1
        case 'CornerAngle':
            match encoded:
                case 0:
                    return nan_value
                case _:
                    return (encoded-1)*ANGLE_INTERVAL + 90.00
        case 'DilationAngle':
            match encoded:
                case 0:
                    return nan_value
                case _:
                    return (encoded-1)*ANGLE_INTERVAL + 61.875
        case 'Length':
            return encoded/10.0
        case 'Width':
            match encoded:
                case 0:
                    return nan_value
                case _:
                    return encoded/10.0
        case 'Height':
            return encoded/10.0 + 0.2


In [6]:
buffer: str = ''
record_size: int = 39       # 39 bits needed to decode one record (line)

def flush_buffer() -> None:
    global data
    global buffer
    while len(buffer) >= record_size:
        data.loc[len(data.index)] = [decode('Brightness', buffer[0:2]),
                                     decode('Orientation', buffer[2:3]),
                                     decode('Stripes', buffer[3:10]),
                                     decode('AntiStripes', buffer[10:14]),
                                     decode('CornerAngle', buffer[14:18]),
                                     decode('DilationAngle', buffer[18:22]),
                                     decode('Length', buffer[22:28]),
                                     decode('Width', buffer[28:34]),
                                     decode('Height', buffer[34:39])] 
        buffer = buffer[record_size:]

with open(file='data.bin', mode='rb') as file:
    byte = file.read(1)
    while byte != b'':
        buffer += bin(ord(byte))[2:].zfill(7)
        flush_buffer()
        byte = file.read(1)

In [7]:
data

,Brightness,Orientation,Stripes,AntiStripes,CornerAngle,DilationAngle,Length,Width,Height
0,1.0,1.0,96.0,3.0,129.375,101.250,3.3,4.1,1.0
1,1.0,1.0,34.0,8.0,123.750,106.875,2.6,3.6,1.0
2,1.0,0.0,28.0,3.0,118.125,95.625,3.0,3.3,1.1
3,1.0,1.0,NaN,7.0,112.500,95.625,1.8,2.3,0.7
4,1.0,1.0,27.0,8.0,140.625,106.875,2.7,3.6,0.8
...,...,...,...,...,...,...,...,...,...
827,0.0,1.0,NaN,0.0,123.750,101.250,2.0,2.8,0.7
828,1.0,0.0,23.0,1.0,123.750,112.500,3.0,3.1,1.3
829,0.0,0.0,NaN,0.0,140.625,123.750,1.4,2.7,0.3
830,1.0,1.0,NaN,NaN,123.750,95.625,2.5,2.1,0.8
